In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

Hyperparameters

In [2]:
input_size = 3  # Number of input features
hidden_size = 50  # Number of LSTM units
output_size = 1  # Number of output features
num_layers = 2  # Number of LSTM layers
num_epochs = 100
learning_rate = 0.001
sequence_length = 10
batch_size = 32

Dataset

In [3]:
class PuckDataset(Dataset):
    def __init__(self, X, y, sequence_length):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.X) - self.sequence_length + 1

    def __getitem__(self, idx):
        return (
            self.X[idx : idx + self.sequence_length],
            self.y[idx + self.sequence_length - 1],
        )
    


Data Loader

In [6]:
df = pd.read_csv("Data/cleaned_data.csv")
X = df[["Puck_cen_X","Puck_cen_Y","direction"]].values # values between 2 and -1
Y = df["Cross_Left"].values
#Y = df["Cross_right"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

# Create instances of the Dataset class
train_dataset = PuckDataset(X_train, y_train, sequence_length)
test_dataset = PuckDataset(X_test, y_test, sequence_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [7]:
print(train_loader)

LSTM Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

Criterion

In [ ]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Training

In [ ]:
for epoch in range(num_epochs):
    outputs = model(train_sequences)
    loss = criterion(outputs, train_labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Testing

In [ ]:
model.eval()
with torch.no_grad():
    test_outputs = model(test_sequences)
    mse = criterion(test_outputs, test_labels)
    print(f'Mean Squared Error on Test Data: {mse.item():.4f}')

Visualization

In [ ]:
plt.plot(test_labels.numpy(), label='Actual')
plt.plot(test_outputs.numpy(), label='Predicted')
plt.legend()
plt.show()